In [2]:
import pandas as pd 
import math
from matplotlib import pyplot as plt
from importlib import reload 
import sys 
sys.path.append( "../")
import dw_utils3

Loading dw_utils3 module (v. 20181113)


In [3]:
ans_submit = dw_utils3.create_submitter(host='52.91.20.10', port=443, 
                       user="your.name@yuxiglobal.com", # put your full yuxi email address here, including @yuxiglobal.com
                       ws_key="dw3", #this is the workshop key, don't change it 
                       token='...' ) #put the token that Mateo sent to you on an e-mail on Wednesday...

Creating submitter function and submitting test question to verify connection.
Q00 = 'test answer'
hostname '52.91.20.10' doesn't match 'data-workshops.yuxiglobal.com'


**IMPORTANT:** Set the path to data to the right value where you put your (e.g. `"C:/Users/username/Downloads/"`)
If you put the data files for this workshop in the same directory as the notebook ('.ipynb' file) then, just set this to `"./"`)

In [6]:
DATA_DIR = "C:/_DATA/DW3/" # change to "./" if you put the data files next to the notebook file.

## Introduction

In this data workshop, we will see some advanced data operatorions using Pandas such as merging, grouping, accessing data through an index, as well as re-arranging data. 

Let's load our 'House Prices' dataset that we are familiar with from the previous workshop:

In [7]:
houses_df = pd.read_csv( DATA_DIR + "house_prices_and_characteristics.csv" )

pd.set_option("display.max_columns", 50)

In [8]:
houses_df.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,...,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,...,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,...,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,...,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,...,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,...,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Group By

As its name  implies, this function groups rows of a data frame according to the values in one or more columns.  However, simply calling the function doesn't actually return a `DataFrame` object but rather a `DataFrameGroupBy` object. The latter can be thought of as a dictionary with its keys being the distinct combinations of grouping column values, and each associated value being the group of rows for a column values combination, assembled as a data frame. On a `DataFrameGroupBy` object, one can then apply aggregation operations, such as sum, mean, mode, etc. Keep in mind, that they will only be applied to columns which support such operation, with the other columns being excluded (i.e. non numerical columns being excluded when trying to get the average for each column of the resulting data frame).



In [9]:
grp_by = houses_df.groupby(["MSZoning", "Street"])
type( grp_by  )

pandas.core.groupby.DataFrameGroupBy

Take a look at the `.groups` attribute of the `grp_by` object. Notice this is just a **dictionary**, with the keys being tuples which are all the different combinations of values from the grouping columns.

Don't be scared by `Int64Index` objects! They are just memory efficent ways to collect the indices of rows in each group...

In [10]:
grp_by.groups

{('C (all)', 'Grvl'): Int64Index([812, 1061], dtype='int64'),
 ('C (all)',
  'Pave'): Int64Index([30, 88, 93, 495, 557, 711, 916, 1279], dtype='int64'),
 ('FV',
  'Pave'): Int64Index([  47,   56,   87,  105,  115,  158,  180,  212,  240,  256,  270,
              281,  285,  297,  317,  377,  381,  399,  409,  412,  453,  460,
              501,  507,  525,  549,  578,  603,  623,  641,  644,  650,  686,
              687,  699,  755,  758,  762,  824,  829,  831,  864,  875,  885,
              914,  959,  973,  975,  977,  989, 1087, 1089, 1091, 1172, 1191,
             1217, 1246, 1265, 1317, 1358, 1364, 1365, 1374, 1442, 1454],
            dtype='int64'),
 ('RH',
  'Pave'): Int64Index([ 341,  383,  543,  635,  671,  681,  778,  840,  913,  951,  955,
             1030, 1206, 1234, 1264, 1326],
            dtype='int64'),
 ('RL', 'Grvl'): Int64Index([335, 582, 1184], dtype='int64'),
 ('RL',
  'Pave'): Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    9,   10,
          

### Groupby Exercise 0

Take `grp_by` and get the number of rows in the group with key `('RH', 'Pave')`

In [12]:
len( grp_by.groups[ ('RH', 'Pave') ] )

16

In [ ]:
num_rows = ... # Replace ... with your code
ans_submit( 'Grp0', num_rows )

### Groupby Exercise 1

How many different streets are there in `houses_df`? (suggestion, do some numerical operation on the groupby)c

In [30]:
num_streets = len(houses_df.groupby(['Street'] )) # Replace ... with your code
ans_submit( 'Grp1', num_streets )

Grp1 = 2
Answer for question Grp1 is correct 😃



### Groupby Exercise 2

How many different combinations of `Street` and `LandContour` are there in `houses_df`?

In [ ]:
num_combinations = ... # Replace ... with your code
ans_submit( 'Grp2', num_combinations )

## Applying a user-defined function to each group in a group-by

The fact that group-by  returns multiple sub-data-frames allows _functions to be applied to each individual group_, rather than on the entire data frame, which is useful when trying to analyze data in different groups without having the results of one group being affected by data from other groups. 

In the following example, two new columns are added to `grp_by` via an apply function, one of them being the rank (position that a value occupies within a sorted list) and the other being the average price of the top half most expensive houses of each group.

In [24]:
df = pd.DataFrame(  { "price" : [ 17, 5, 23, 8, 2, 4] } )
df
df["rank_price"] = df["price"].rank()
df.sort_values('price')

,price,rank_price
4,2,1.0
5,4,2.0
1,5,3.0
3,8,4.0
0,17,5.0
2,23,6.0


In [28]:
df['top_half'] = df['rank_price'] < (len(df) / 2) 
df

,price,rank_price,top_half
0,17,5.0,False
1,5,3.0,False
2,23,6.0,False
3,8,4.0,False
4,2,1.0,True
5,4,2.0,True


In [27]:
df['price'][  df['rank_price'] < (len(df) / 2)  ]

4    2
5    4
Name: price, dtype: int64

In [29]:
df['price'][  df['rank_price'] < (len(df) / 2)  ].mean()

3.0

In [ ]:
def enrich_grp( grp ) : 
    """Enrich a group (sub-data-frame within a DataFrameGroupBy) by adding a column that ranks the SalePrices in 
    it and then compute the avg price of the most expensive half of houses. Notice that this number is going to 
    be the same accross all rows! The length (number of rows) of the resulting data frame is the same as the input one"""
    
    grp['RankPrice'] = grp['SalePrice'].rank()
    grp['avg_price_top_half'] = grp["SalePrice"][ grp["RankPrice"] < len(grp) / 2 ].mean()  
    
    return grp

Now we pass the function above to the 'higher-order' method `.apply`! 
This is sort of like running a loop over the groups but in a more elegant, maintanable way,  and without writing `for` or `while`.

In [ ]:
( grp_by.apply( enrich_grp )    
        [["SalePrice", "RankPrice", "avg_price_top_half"]]
        .sort_values( ['avg_price_top_half', 'RankPrice']) ).head(20)

Notice that the value of 'avg_price_top_half' is constant within each group but, unfortunately, the group identication information is lost...

One way to include in the result is the following 'trick':

In [ ]:
def enrich_grp_v2( grp ) : 
    """Enrich a group (sub-data-frame within a DataFrameGroupBy) by adding a column that ranks the SalePrices in 
    it and then compute the avg price of the most expensive half of houses. Notice that this number is going to 
    be the same accross all rows! The length (number of rows) of the resulting data frame is the same as the input one"""
    
    # TRICK to retrieve the v
    # Access each component of the name attribute of the group to find out the original values 
    # of the grouping columns
    grp['MSZoning'] = grp.name[0] 
    grp['Street']   = grp.name[1]
    
    
    grp['RankPrice'] = grp['SalePrice'].rank()
    grp['avg_price_top_half'] = grp["SalePrice"][ grp["RankPrice"] < len(grp) / 2 ].mean()  
    
    return grp

In [ ]:
( grp_by.apply( enrich_grp_v2 )    
        [["MSZoning", "Street", "SalePrice", "RankPrice", "avg_price_top_half"]]
        .sort_values( ['avg_price_top_half', 'RankPrice']) ).head(20)

### Groupby Exercise 3

Redo the same grouping of the previous exercise but instead of getting the sum of lot areas, create a function that gets the _second_ largest lot area in the group.  In case the group has only 1 row, return float("NaN")

** Hints: **
 * You can sort a series (column), such as `grp['LotArea']` in descending order by calling `.sort_values(ascending=False)` on it. 
 * You can find out the number of rows in a df (or grp) by calling `len()` on it (this is a function, not method) 
 * You can get the second element of a (sorted) series `ser`  with the syntax `ser.iloc[1]`

In [ ]:
# replace ... with your code 
def second_largest( grp ) :
    if len( grp ) > 1 : 
        sorted_areas =  grp['LotArea'].sort_values( ascending=False )
        return sorted_areas.iloc[1]
    else :
        return float( "NaN" )

serG3 = houses_df.groupby( ["MSZoning", "Street"] ).apply( second_largest )  
serG3

In [18]:
serG3.sum()

252766.0

In [ ]:
ans_submit( 'Grp3', int(serG3.sum()) )  

## Aggregation

The aggregation method `.aggregate` (_alias_: `.agg`)  in the class `DataFrameGroupBy` allows multiple aggregation operations to be performed on all or a subset of the columns of data frame

This functions admits two ways of calling it: 

In the first one *a list of aggregation function names* (strings) is passed to it.  This results in all those aggregation functions being applied to all columns of the dataframe (where the aggregation functions applies):

In [ ]:
grp_by.aggregate(["sum", "mean"])   # effectively the same as houses_df.groupby( ... ).aggregate( ["sum", "mean"] )

The operations are only applied on columns which allow them, while the rest of columns are simply ignored from the output. In the following example it can be seen how `aggregated` ends up having less columns that the data frame it came from, `houses_df`, due to the latter having columns which don't allow numerical operations to be performed.

In [ ]:
aggregated = houses_df.groupby(["MSZoning", "Street"]).mean()
print("houses_df has %d columns\naggregated has %d columns" % (houses_df.shape[1],aggregated.shape[1]))

A second way of calling it is passing a dictionary in which the keys are data frame columns and the values are the names of aggregation functions to apply to each column and on each group, separately.

In [ ]:
grp_by.agg({ "LotArea"     : "mean",
             "SalePrice"   : "sum",
             "MSSubClass"  : pd.np.median,    
             # Series.nunique() computes the number of unique values in a series. 
             "MiscVal"     : lambda ser : ser.nunique(),
             # Series.value_counts() returns dict of counts of different values
             "Exterior1st" : lambda ser : " | ".join( ser.value_counts().keys() )   }) 

It is also possible to aggregate after group-by, without calling `.aggregate`, one can directly call a particular aggregation (e.g. `.sum()`, `.mean()`, `.count()` and several others) method to all 'compatible' compatible columns. The advantage / disadavantage of is that the same method and only that one is applied to all columns

In [ ]:
houses_df.groupby(["Street", "LandContour"]).count()

**And now for the actual aggregation exercises, in which the output data does matter**

### Aggregation Exercise 1

Group `houses_df` by `Street` and `LotShape` and sum only the lot area of each group

In [ ]:
ser0 = ... # Replace ... with your code
ser0

In [ ]:
ans_submit( 'Agg1', ser0.loc[('Grvl', 'Reg')]['LotArea'] )  #In a section below we explain what .loc does.

## Introducing the Gowalla check-ins dataset

In the following examples we shall work on (a subset of) the Gowalla check-ins dataset. 

This dataset contains check-in events by users of the Gowalla social networking site to set of locations. 

In [ ]:
checkins_df = ( pd.read_csv( DATA_DIR + "check_in_gowalla.csv")                  
                  .rename( columns={"check_in_time" : "checkin_time", 'location_id' : "location" })
                  .sort_values( 'location' ))

checkins_df.head(10) 

### Aggregation Exercise 3

Use `checkins_df` to get the number of visits and datetime of the first visit for each location

In [ ]:
agg3_df = ... # replace ... with your code 
ans_submit( 'Agg31',agg3_df['user'].sum())
ans_submit( 'Agg32',agg3_df['checkin_time'].loc[9101])

### Aggregation Exercise 4

Use `checkins_df` to compute the number of unique (different) visitors per location (suggestion: search for the `nunique` method), for all locations at once!

In [ ]:
agg4_df = ...  # replace ... with your code 

ans_submit( 'Agg4',agg4_df['user'].loc[9101])

## Indexing

An **index** in a data frame is a set of values that play essentially the same role as the keys in a dictionary. An index is usually composed of integers, strings or tuples. To each value of the index, there is an associated row or, possibly, a *set of rows* in the data frame. 

Further, accessing that row or set of rows is an efficient (O(1)) operation that *does not* require to scan the whole data set. 

It's important to note that the index is *not considered* a column in the dataset as such. It's in a different category. However, an index can be easily turned into a column as we will see below.

### set_index

Every data frame has an index. When the dataframe is first created from scratch, the index keys are just integers ranging from 0 to `len(df)-1`. 

However it is often more useful to define and index from the values of one or several columns column.
This is done by means of the `.set_index` method (in the `DataFrame` class)

This method makes turns the specified column into the new index of the data frame by either taking it out of the data columns (default) or keeping it both as the index as well as a data column (by setting the drop parameter to False). More than one column may be set as index by inputing a list of columns.

Remember the `checkins_df` looks like

In [ ]:
checkins_df.head(15)

Notice that there is a column of apparently random numbers  on the let that is not named. That's actually not a column, but the index. Notice that, when we defined the dataframe, we sorted it by `location` right after loading it from the csv. The first record in the resulting order by `location` actually comes from the 826-th row of the `csv`. That's where the 826 in front of the first record comes from.  

Looking at the `.index` attribute we see that `checkins_df` has an index consisting of integers, in the order just shown.

In [ ]:
checkins_df.index 

In [ ]:
chins_indexed_1 = checkins_df.set_index( 'location' )
chins_indexed_1.head( 20 )

In [ ]:
chins_indexed_1.index

Again, the index consists of integers but they come from `location` column in the original data frame.

It's important to see that setting an index on a data frame doesn't actually change the original data frame at all, rather it creates a new one with the specified index. 

In [ ]:
checkins_df.head(10)

Not let's define and index on two columns!

In [ ]:
chins_indexed_2 = checkins_df.set_index( ['location', 'user'] )
chins_indexed_2.head( 20 )

The resulting index is called a _hierarchical_ index because it has a hierarchy of levels. 

In this case , the first level groups  records by location an the second by user.

** Exercise I0: ** 

The following line generates an error. Copy an paste the *last* (non-empty) line of the error message into the answer. 
Make sure to understand **why** this error is produced. If you don't understand, discuss it with your instructor. 

In [ ]:
chins_indexed_1['location']

In [ ]:
ans_submit( "I0", "copy and paste the last (non-empty) line of the error message")

It is also possible to keep a column as data as well as setting it as and index by  passing the `drop = False` to `set_index`.

In [ ]:
chins_indexed_1 = checkins_df.set_index("location", drop=False)
chins_indexed_1.head( 20 )

### Retrieving rows through an index -- basic usage


The primary purpose of an index is to *efficiently* retrieve a row or set of rows from a data frame. This is done throught the `DataFrame.loc` accessor ( 'loc' is short for 'locate' ). We will go in depth into the usage of `loc`, but for now the essential usage is as follows:

In [ ]:
first_row = checkins_df.loc[826]
first_row

** Exercise I1: ** 

What is the _type_ of `first_row`?  (You can use the built-in function `type()` to answer this question)

In [ ]:
ans_submit( "I1", "pandas.....fill-in rest of the type (fully-qualified) name")

In [ ]:
first_row_v2 = checkins_df.loc[ [826] ]
first_row_v2

This is the same as before except that it is nicely formatted (why?)


**Exercise I2**

What is the type of `first_row_v2`? 

In [ ]:
ans_submit( "I2", "pandas.....")

In [ ]:
another_row = checkins_df.loc[ [0] ]
another_row

Notice that a *single* index value can map to _many_ rows...

In [ ]:
chins_indexed_1.loc[1038401]

** Exercise I3 **

How many rows are there for location = 9501

In [ ]:
ans_submit( "I3",...  ) # your answer instead of ...

### Accessing rows when there is a hierarchichal index 

When there are _n_ levels in the index, you can specify 1, 2 or up to 'n' values in to the `.loc` accessor 

In [ ]:
chins_indexed_2.loc[ 9101 ]

In [ ]:
chins_indexed_2.loc[  (9101, 346 ), ]

In order to avoid that nasty warning we are careful to sort both levels of the index..

In [ ]:
chins_indexed_2s = chins_indexed_2.sort_index( level= [0,1], axis=0)  
chins_indexed_2s.loc[ (9101,346) ]

### reset_index

It reset the index to an auto-incremental one and makes any previously set index a data column again. By default it resets all previously indexed columns, but the parameter level allows only certain indexes to be reset

In [ ]:
chins_indexed_2.reset_index().head(20)

In [ ]:
chins_indexed_2.reset_index(level=1).head(20)

## Exercise R0

Reset the index of `chins_indexed_2s` by one level and count how many different users have been on location 9101

In [ ]:
r0_ans = ...  # replace ... with your answer
ans_submit( 'R0', r0_ans  )

## iloc and loc

Pandas' main way of referencing data is through `.iloc` and `.loc` _accessors_ (which are special types of attributes)

Both accesors are similar but only superficially so.

`iloc` is simpler, it's basic usage syntax is as follows

```
   df.iloc[ list_of_ints_r, list_of_ints_c ]  
```

Here `list_of_ints_r` specifies the 0-based _integer indices_ of _rows_ in the dataframe's own order and and `list_of_ints_c` specifies the 0-based indices of `columns`. Despite its name `iloc` does not take into account the dataframe's index at all! The `i` in `iloc` stands for `integer`. 

Instead of `list_of_ints_r`, you can also put a slice, such as `10:47` to get rows numbered 10 through 46 (not including 47 as is usual with Python slicing). Remember that just writing `:47` is a shorthand for `0:47`  and `10:` is a shorthand for `10:len(df)`. Similarly just writing `:` means `0:len(df)`. 

Analogous considerations hold for the second argument, `list_of_ints_c`. 


Now onto `loc`.

The `.loc` method (accessor)  receives as input list of row keys and another one column names.

Row keys are the keys defined in the index. The colum names are the regular column names you already know and love. 


The basic syntax is:

```
   df.loc[ list_of_row_keys, list_of_col_names ]  
```

It is also possible to  specify the _row keys_ only:

```
   df.loc[ list_of_row_keys ]  
```



In [ ]:
houses_df2 = houses_df.set_index("Neighborhood")
houses_df2.head(10)

In [ ]:
houses_df2.iloc[ [1,2,3], [0, 1,3] ]  # just indexing by the order they appear in the df. 0-based indexing!

In [ ]:
houses_df2.iloc[ 1:7, 0:5 ]

Now we use `.loc` with only row keys: 

Notice that a **single key can refer to many rows**! (and this is not unusual...)

In [ ]:
houses_df2.loc[ ["Blueste","NPkVill"] ].head(20)

In [ ]:
houses_df2.loc[ ["Blueste","NPkVill"], ['LotArea', 'Street'] ].head(20)

Turn index back into a column

In [ ]:
houses_df2 = houses_df2.reset_index()
houses_df2.head( 10 )

## Pivot

The pivot function generates a new dataframe rearranging a column from a given data  frame in the form of a grid, which may be clearer to analyze. 

The function receives three basic arguments:

  * **index**: specifies the column whose values will become the index of the new data frame.
  * **columns** : a column whose values are going to be turned into column headers of the new data frame. It is advisable for this particular column not to have many different values so that the new data frame won't be too wide (have a lot of columns).
  * ** values ** : A column whose values are going to be displayed in the grid. Although a list of columns can be used for values , doing so would be the same as simply pivoting 2 different values and concatenating the result.
  
All this will be a lot clearer with an example. 

First we generate an aggregated version of our houses data frame to get average prices and areas for all the various combinations of Exterior1st and Street.

In [ ]:
houses_agg = ( houses_df.groupby(["Exterior1st", "Street"])
                        .agg( {"SalePrice" : "mean", "LotArea" : "mean"})
                        .reset_index()
                        .sort_values('Street')) 
houses_agg

Notice that only some of the values of 'Exterior1st' appeared for `Street = 'Grvl'`, hence the grid contains NaNs for those.  

In [ ]:
houses_agg.pivot(index='Exterior1st', columns="Street", values='SalePrice').head(10)

## Pivot Exercise 0

Take the houses_df data frame, group it by MSZoning and LotShape, get the average SalePrice and pivot it with LotShape and MSZoning as index and columns respectively (to use either LotShape or MSZoning to pivot, you have to reset the index of the group by)


In [ ]:
# replace ... with your code :
houses_agg = ...
df = ...  
df 

In [ ]:
ans_submit( 'Piv0',df.iloc[1,2]  )

## Merge and join

Both functions allow data from different data frames to be combined into a single one according to a 'crossing' or 'look-up' rule. 

Although both `merge` and `join` do really a similar things, the way they do them is different. 

The function `merge` is pandas default function for joining data. It's basically *pandas counterpart of SQL's join*, and requires the specification of which columns of both data frames would be compared. Merge doesn't care at all about about the indexes defined on them. 

On the other hand, Panda's `join function is more of a convenience thing (it even uses merge internally), joining is basically doing a merge by taking advantage of the indexes of both data frames. 

The following figure summarizes the different 4 types or merge: _ inner, outer, left and right_.

The function merge is also availaible as a method in the  `DataFrame` class. 
The basic syntax is:

```
new_joined_df = df.merge( another_df, left_on = "col_in_df",  right_on = "col_in_another_df",
                          how="inner"|"left"|"right"|"outer" ) 
```

The first argument (`another_df`) as well as `left_on` and `right_on` are required arguments. 
`left_on` specifies a column name on the data frame `df` whose values should be matched with 
those of the  `another_df`'s column specified by `right_on` in. 

The `how` argument is optional ans specifies the type of join:
 
 <img src="merge.png" height="200" width="800"/>

To see some examples, we load some (financial!) data first.

In [ ]:
usdcop = pd.read_csv( DATA_DIR + 'usdcop.csv', delimiter = "\t", infer_datetime_format=True)
usdcop

In [ ]:
usdcop = pd.read_csv( DATA_DIR + 'usdcop.csv', delimiter = "\t", infer_datetime_format=True)
usdcop['usd_cop'] = usdcop['usd_cop'].str.replace('$', '').str.replace(',', '').astype( float )
usdcop['fecha'] = ( usdcop['fecha'].str[6:10] + '-' + usdcop['fecha'].str[3:5] + '-' + usdcop['fecha'].str[0:2] )
usdcop  # Now contains the exchange rate from usd to cop  as type float!

In [ ]:
btcusd = pd.read_csv( DATA_DIR + 'btcusd.csv' ) # this file contains hourly exchange rate from BTC (bitcoin) to USD
btcusd['date'] = btcusd['date_tm'].str[0:10]
btcusd['time'] = btcusd['date_tm'].str[11:]

btcusd.head(10) 

In [ ]:
btcusd_day_max = btcusd.groupby('date').agg( {'btc_usd' : 'max' } ).reset_index()
btcusd_day_min = btcusd.groupby('date').agg( {'btc_usd' : 'min' } ).reset_index()
btcusd_day = pd.concat( [btcusd_day_max, btcusd_day_min] ).sort_values( 'date' )
btcusd_day  # Now contains both the minimum and maximum exchange rates between btc (bitcoin) and USD

## Inner merge

Is the default merge in case the `how` parameter is not specified. It yields rows for which there are matching values of the specified merge columns on both data_Frames.

In [ ]:
usdcop.merge( btcusd_day_min, left_on='fecha', right_on='date' )

**Question:** Why did the first `btc_usd` price (7930.79) get duplicated?  If you don't understand it, discuss it with your instructor! 

In [ ]:
usdcop.merge( btcusd_day, left_on='fecha', right_on='date' )

## Left merge

It gkeeps all the data from the first data frame, adding data from the second one whenever there is a row matching and filling with `NaN` the missing columns from the second data frame in which no match was found

In [ ]:
usdcop.merge( btcusd_day_min, how="left", left_on='fecha', right_on='date')

## Right merge

It's pretty much the same exact thing as left merge with the data frame on the left taking being on the right and viceversa. It keeps all the data from the second data frame adding data from the first one whenever they intersect and filling with `NaN` the missing columns from the first data frame if no match was found

In [ ]:
usdcop.merge( btcusd_day_min,  how="right", left_on='fecha', right_on='date').sort_values( 'date' )

## Outer merge

It's basically the combination of both left and right join, keeping all the data from both data frames and filling out with NaN if no match found was found for either.

In [ ]:
usdcop.merge( btcusd_day_min, how="outer", left_on='fecha', right_on='date')

### Merge Exercise 0 

Compute the outer merge between the `usdcop` and `btcusd_day` data frames on the `fecha`/`date`. Call the resulting data frame `buc_min`. To this data frame, add a new column `btc_cop` containing the exchange rate from `btc` to `cop`. This can be computed as the product of the two available exchange rates (`btc_usd`, `usd_cop`). 

Now generate a new series `agg` by applying `groupby('date')` to `buc_min` and compute both  the mean and the median of the new column for each `date`.


In [ ]:
# replace ... by your code 
buc_min = ... 
...
agg = buc_min.groupby('date').agg( {"btc_cop" : ["mean", "median"]} ) 

ans_submit( 'Mer0', int(agg[('btc_cop', 'median')].sum()) )

### Join

Sometimes, when data frames are already indexed by the same 'thing', it is a lot easier to use `join`, which uses matches rows from two them according to their index value. 

To see this, let's define two dataframes indexed by 'location' 

In [ ]:
counts_by_loc = ( checkins_df.groupby( "location" )
                             .agg( { "user" : "nunique",
                                     "checkin_time" : "count"}) )
counts_by_loc.head(10)

In [ ]:
lat_lon_by_loc = ( checkins_df[['location', 'latitude', 'longitude']]
                              .drop_duplicates()
                              .set_index('location') ) 
lat_lon_by_loc.head(10)

As the both dataframes are indexed on the same id, it is very easy to just join them by that id.

In [ ]:
counts_by_loc.join( lat_lon_by_loc ).head(10)

A non so conventional way of merging data is by merging a data frame with itself as to compare some rows with one another, here's an example of it.

### A non-conventional merge: self-merge

In [ ]:
chins2 = checkins_df.merge( checkins_df, on="location" )
chins3 = chins2[ (chins2.checkin_time_x < chins2.checkin_time_y) & 
                 (chins2.user_x != chins2.user_y )]
chins4 = ( chins3[["user_x", "user_y", "location"]]
                 .drop_duplicates()                 
                 .groupby(["user_x", "user_y"])
                 .agg( {"location" : "count"})
                 .rename( columns = { "location" : "location_count" } )
                 .reset_index() )

chins4.sort_values('location_count', ascending=False).head(40)

Here we merged the checkins data frame with itself every time an user went to a location some other user had gone as well. We then filter the rows in which an user arrived after another one, not at the same time (given that the previous merge rows matched with themselves). Finally the information was grouped for a better visualization.

### Merge Exercise 1

Perform an inner merge between the first 100 rows and the following 100 rows of `houses_df` comparing the `Neighborhood` column.
Call the result `houses_df_self_join`. How many rows does this df have?

In [ ]:
houses_df_self_join = ... # replace ... with your code 
ans_submit( 'Mer1', ... )   

## Shift

The shift function is pandas' way of lagging a series, which is useful when trying to see how something changes as time goes on, like predicting a future value for certain series, or studying how a given series affects another one

In [ ]:
chins_u4771 = checkins_df[ checkins_df.user == 4771 ].copy()
chins_u4771.head(15)

In [ ]:
chins_u4771['checkin_tm_prev'] = chins_u4771['checkin_time'].shift(1)
chins_u4771['checkin_tm_next'] = chins_u4771['checkin_time'].shift(-1)
chins_u4771.head(15)

In the previous example the shift function was really poorly used, given that by simple shifting every register by one position the first register for every user (apart from user 0) is literally the last register from the previous user which is obviously false information. To avoid doing so, it would be best to first group up the data by user, that being said...

## Shift Exercise 0

Shift only the check in time of every user in the `checkins_df` by 1 position (as a sugestion do a groupby and apply the same technique as with enrich_grp function above...) 

In [ ]:
df = ... # replace ... with your code 
ans_submit( 'Shift0', math.isnan(df.iloc[2,-1])  )